# Рекомендация тарифов телеком компании

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Первый-взгляд" data-toc-modified-id="Первый-взгляд-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Первый взгляд</a></span></li><li><span><a href="#Разделение-данных-на-выборки" data-toc-modified-id="Разделение-данных-на-выборки-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Разделение данных на выборки</a></span></li><li><span><a href="#Исследование-моделей" data-toc-modified-id="Исследование-моделей-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Исследование моделей</a></span><ul class="toc-item"><li><span><a href="#Решающее-дерево" data-toc-modified-id="Решающее-дерево-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Решающее дерево</a></span></li><li><span><a href="#Случайный-лес" data-toc-modified-id="Случайный-лес-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Случайный лес</a></span></li><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Логистическая регрессия</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Проверка-модели-на-тестовой-выборке" data-toc-modified-id="Проверка-модели-на-тестовой-выборке-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка модели на тестовой выборке</a></span><ul class="toc-item"><li><span><a href="#Решающее-дерево" data-toc-modified-id="Решающее-дерево-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Решающее дерево</a></span></li><li><span><a href="#Случайный-лес" data-toc-modified-id="Случайный-лес-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Случайный лес</a></span></li><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Логистическая регрессия</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-4.4"><span class="toc-item-num">4.4&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Проверка-модели-на-адекватность" data-toc-modified-id="Проверка-модели-на-адекватность-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Проверка модели на адекватность</a></span></li><li><span><a href="#Общий-вывод" data-toc-modified-id="Общий-вывод-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Общий вывод</a></span></li></ul></div>

## Первый взгляд

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


- сalls — количество звонков,
- minutes — суммарная длительность звонков в минутах,
- messages — количество sms-сообщений,
- mb_used — израсходованный интернет-трафик в Мб,
- is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

Исходные данные представляют собой таблицу на 5 столбцов и 3214 строк. Среди них нет ни одного пропуска, а типы данных по большей части соответствуют ожидаемым. Исключение составляют столбцы `calls` и `messages`, содержащие количество звонков и сообщений - у них тип данных float, хотя в них явно должны быть целочисленные значения. Впрочем, нам это никак не мешает. Если верить условиям задачи, то предобработка уже выполнена, и не стоит ожидать встретить в данных выбросы и неадекватные значения.

## Разделение данных на выборки

Прежде всего выделим целевой признак, отделив столбец `is_ultra` от всей остальной таблицы:

In [3]:
X = df.drop('is_ultra', axis=1)
y = df['is_ultra']

Теперь разделим данные на обучающую, валидационную и тестовую выборки. В нашем распоряжении всего 3214 объектов, поэтому для обучения модели выделим 70% всех данных, а оставшиеся 30% поделим поровну на валидационную и тестовую выборки:

In [4]:
X_train, X_check, y_train, y_check = train_test_split(X, y, 
                                                      test_size=.3, 
                                                      random_state=123, 
                                                      stratify=y)
X_valid, X_test, y_valid, y_test = train_test_split(X_check, y_check, 
                                                    test_size=.5, 
                                                    random_state=123, 
                                                    stratify=y_check)

In [12]:
list = [[X_train, X_valid, X_test], 
        [y_train, y_valid, y_test], 
        ['X_train', 'X_valid', 'X_test'], 
        ['y_train', 'y_valid', 'y_test']]

for i in range(2):
    for y in range(3):
        print(f"Размер выборки {list[i+2][y]}: {len(list[i][y])}")

Размер выборки X_train: 2249
Размер выборки X_valid: 482
Размер выборки X_test: 483
Размер выборки y_train: 2249
Размер выборки y_valid: 482
Размер выборки y_test: 483


Мы получили три выборки - обучающую(`X_train`, `y_train`), валидационную(`X_valid`, `y_valid`) и тестовую(`X_test`, `y_test`), разделяющие исходные данные в соотношении 70/15/15.
Чтобы соотношение классов целевого признака в выборках сохранилось таким же, каким было в исходных данных, был использован параметр `stratify` со значениями `y` и `y_check`.

## Исследование моделей

Обучим три модели, после чего проверим их на валидационной выборке и выведем на экран значение метрики accuracy. Это позволит нам сравнить эффективность моделей, и выбрать ту, которую затем проверим на тестовой выборке.

### Решающее дерево 

In [5]:
best_accuracy = 0
best_depth_train_dtc = 0

for depth in range(1, 15):
    model = DecisionTreeClassifier(max_depth=depth, random_state=123)
    model.fit(X_train, y_train)
    accuracy = model.score(X_valid, y_valid)
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_depth_train_dtc = depth

print('Наивысшая точность:', best_accuracy, 'при глубине дерева', best_depth_train_dtc)

Наивысшая точность: 0.7842323651452282 при глубине дерева 9


В цикле перебирается 15 моделей, с глубиной дерева от 1 до 15. На валидационной выборке лучше всего себя показала глубина дерева 9 - точность составила 0.78.

### Случайный лес 

In [6]:
best_accuracy = 0
best_depth_train_rfc = 0
best_est_train_rfc = 0

for est in range(1, 50):
    for depth in range(1, 15):
        model = RandomForestClassifier(max_depth=depth, n_estimators=est, random_state=123)
        model.fit(X_train, y_train)
        accuracy = model.score(X_valid, y_valid)
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_depth_train_rfc = depth
            best_est_train_rfc = est

print('Наивысшая точность:', best_accuracy, 
      'при глубине дерева', best_depth_train_rfc, 
      'и количестве деревьев', best_est_train_rfc)

Наивысшая точность: 0.8008298755186722 при глубине дерева 11 и количестве деревьев 6


В случае со случайным лесом мы перебрали 750 моделей, изменяя только два гиперпараметра: количество деревьев и их глубину. Из этого ассортимента лучший результат на валидационной выборке показала модель из 6 деревьев с глубиной 11 - accuracy составила 0.80.

### Логистическая регрессия 

In [7]:
best_accuracy = 0
best_solver = ''

solver =['newton-cg', 'lbfgs', 'liblinear']
for i in range(3):
    model = LogisticRegression(solver=solver[i], random_state=123)
    model.fit(X_train, y_train)
    accuracy = model.score(X_valid, y_valid)
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_solver = solver[i]
print('Наивысшая точность:', best_accuracy, 'при значении solver =', best_solver)

Наивысшая точность: 0.7634854771784232 при значении solver = newton-cg


/opt/conda/lib/python3.7/site-packages/scipy/optimize/linesearch.py:466: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/opt/conda/lib/python3.7/site-packages/scipy/optimize/linesearch.py:314: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)


Последняя модель, логистическая регрессия, проверялась с изменением гиперпараметра `solver`. Наилучшего результата удалось достичь с алгоритмом `'newton-cg'`, точность предсказаний такой модели на валидационной выборке составила 0.76, что меньше, чем на всех остальных моделях.

### Вывод 

Мы обучили и проверили на валидационной выборке три разных модели: решающее дерево, случайный лес и логистическая регрессия. Для каждой из них перебрали несколько гиперпараметров, чтобы найти наиболее эффективную конфигурацию. В итоге мы получили три модели, которые показали самое лучшее значение accuracy на этой выборке. Однако, разница между этими результатами не такая уж значительная - от 0.76 до 0.80.

## Проверка модели на тестовой выборке

Точность предсказания на валидационной выборке у всех моделей отличается довольно слабо, поэтому проверим на тестовой выборке их все. 

### Решающее дерево 

In [15]:
model = DecisionTreeClassifier(max_depth=9, random_state=123)
model.fit(X_train, y_train)
accuracy = model.score(X_test, y_test)

print('Точность предсказаний решающего дерева на тестовой выборке:', accuracy)

Точность предсказаний решающего дерева на тестовой выборке: 0.8095238095238095


Для глубины решающего дерева мы взяли лучшее значение из прошлой проверки. На тестовой выборке модель показала результат получше, чем на валидационной: точность составила 0.80 против 0.78.

### Случайный лес

In [17]:
model = RandomForestClassifier(max_depth=11, n_estimators=6, random_state=123)
model.fit(X_train, y_train)
accuracy = model.score(X_test, y_test)
print('Точность предсказаний случайного леса на тестовой выборке:', accuracy)

Точность предсказаний случайного леса на тестовой выборке: 0.8178053830227743


Как можно заметить, эта модель тоже неплохо справилась. Точность снова оказалась выше, чем на валидационной выборке. Можно сказать, что эта модель достаточно стабильна, и пока это лучший показатель.

### Логистическая регрессия

In [18]:
model = LogisticRegression(solver='newton-cg', random_state=123)
model.fit(X_train, y_train)
accuracy = model.score(X_test, y_test)
print('Точность предсказаний логистической регрессии на тестовой выборке:', accuracy)

Точность предсказаний логистической регрессии на тестовой выборке: 0.7577639751552795


/opt/conda/lib/python3.7/site-packages/scipy/optimize/linesearch.py:466: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/opt/conda/lib/python3.7/site-packages/scipy/optimize/linesearch.py:314: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)


У логистической регрессии на тестовой выборке точность предсказаний составила всего 0.75, и это самый грустный результат. Он даже хуже, чем значение на валидационной выборке.

### Вывод


Из всех изученных вариантов наилучшие результаты удалось получить от моделей случайного леса, как и на валидационной выборке.  
Гиперпараметры модели -  6 деревьев с максимальной глубиной 11, эта конфигурация стабильно дала точность более 0.80 на всех выборках.

## Проверка модели на адекватность

Для проверки того, насколько адекватно полученные модели выполняют свою задачу, и не выдают ли они просто набор псевдослучайных чисел, сравним их значения точности с аналогичным от простейшей фиктивной модели, предсказывающей наиболее часто встречающийся класс.

In [12]:
dummy_clf = DummyClassifier(strategy="most_frequent", random_state=123)
dummy_clf.fit(X_train, y_train)
print('Валидационная выборка:', dummy_clf.score(X_valid, y_valid))
print('Тестовая выборка:', dummy_clf.score(X_test, y_test))

Валидационная выборка: 0.6929460580912863
Тестовая выборка: 0.6935817805383023


Полученное значение ниже, чем точность любой из использованных ранее моделей, из чего можно сделать вывод, что модели, которые мы отобрали, в достаточной степени адекватны.

##  Общий вывод

Мы обучили несколько разных моделей с разными наборами гиперпараметров. Самые высокие значения accuracy, которые удалось получить, составляют 0.80 на валидационной и 0.81 на тестовой выборках, и оба этих значения были получены от модели "случайный лес".   
Однако, возможно, стоит несколько изменить подход к решению задачи, поскольку предлагать пользователям тариф, который им подойдёт с вероятностью 80% как-то не очень корректно, точность прогноза стоит увеличить. Возможно, имеет смысл попробовать другие модели, взять бóльшую исходную выборку, обучить несколько моделей и взять среднее значение, или же разным моделям раздать разные наборы признаков. 